In [1]:
from iduedu import get_intermodal_graph
import geopandas as gpd

# Importing the necessary libraries:
# - 'get_intermodal_graph' from 'objectnat' to retrieve a multimodal transportation network graph
# - 'geopandas' for handling geospatial data

G_intermodal = get_intermodal_graph(osm_id=1114252)
# Retrieving the intermodal graph (public transport, roads, etc.) for a city or region using the OpenStreetMap (OSM) ID 1114252.

2025-03-20 16:54:30.836 | INFO     | iduedu.modules.drive_walk_builder:get_walk_graph:216 - Downloading walk graph from OSM, it may take a while for large territory ...


Parsing public transport routes:   0%|          | 0/61 [00:00<?, ?it/s]

Calculating the weights of the walk graph:   0%|          | 0/48662 [00:00<?, ?it/s]

2025-03-20 16:54:42.551 | INFO     | iduedu.modules.pt_walk_joiner:join_pt_walk_graph:50 - Composing intermodal graph...
2025-03-20 16:54:44.122 | WARNING  | iduedu.modules.pt_walk_joiner:join_pt_walk_graph:179 - Weakly connected components detected. 2 graph islands were removed. These are probably pt routes that are not connected to walking routes. You may try adjusting the max_dist parameter, but this can lead to incorrect data.


In [2]:
from objectnat import get_isochrone_zone_coverage

services = gpd.read_parquet('examples_data/services.parquet').to_crs(32636).iloc[:5]
# Loading service location data from a .parquet file and reprojecting it to EPSG:32636 (UTM zone).
# Only the first 5 services are selected for analysis.

isochrones, stops, routes = get_isochrone_zone_coverage(
    services=services,
    weight_type="time_min",
    weight_value=10,
    city_graph=G_intermodal,
)
# Calculating isochrone zones for the selected services.
# 'weight_type' specifies the type of weight to use (e.g., 'time_min' for travel time in minutes),
# and 'weight_value' (10) indicates the maximum travel time to calculate the isochrones for.
# 'city_graph' is the multimodal transportation graph used to calculate travel times and reachability.
# The function returns:
# - 'isochrones': the zones that can be reached from the services within the given time threshold
# - 'stops': transit stops within those isochrones, if any.
# - 'routes': the transit routes that fall within or pass through the isochrone zones, if any.


2025-03-20 16:55:48.175 | INFO     | objectnat.methods.isochrones:get_accessibility_isochrones:69 - Calculating isochrones distances...
2025-03-20 16:55:48.251 | INFO     | objectnat.methods.isochrones:get_accessibility_isochrones:114 - Building isochrones geometry...


In [3]:
isochrones.explore()

In [4]:
m = isochrones.explore(column='id', categorical=True, tiles='CartoDB positron')
# Visualizing the isochrone zones on an interactive map using 'explore'. 
# 'id' is used as the categorical identifier to color different isochrones, and 'CartoDB positron' is used as the base map.
stops.explore(m=m)
# Overlaying the transit stops on the same map for visualization.
routes.explore(m=m, column='type')
# Overlaying the transit routes on the map, with the 'type' column used to distinguish different route types (e.g., bus, tram).